In [43]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config("spark.ui.port", "0"). \
    enableHiveSupport(). \
    appName("Spark dataSkew Window Dense_Rank"). \
    master("yarn"). \
    config('spark.executor.instances','5'). \
    config('spark.executor.memory','512MB'). \
    config('spark.executor.cores','4'). \
    config('spark.dynamicAllocation.enabled','False'). \
    getOrCreate()

In [55]:
spark.stop()

In [49]:
spark.sparkContext.applicationId

'application_1745651200635_18450'

In [44]:
 #Disable AQE and AdvisoryPartitionSize

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)


In [45]:
# Read Employee Skew data
schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = spark.read.format("csv").schema(schema).option("header", True).load("Datasets/employee_records_skew.csv")

In [38]:
emp.show()

+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+
|first_name| last_name|           job_title|       dob|               email|               phone|  salary|department_id|
+----------+----------+--------------------+----------+--------------------+--------------------+--------+-------------+
|   Richard|  Morrison|Public relations ...|1973-05-05|melissagarcia@exa...|       (699)525-4827|512653.0|            8|
|     Bobby|  Mccarthy|   Barrister's clerk|1974-04-25|   llara@example.net|  (750)846-1602x7458|999836.0|            7|
|    Dennis|    Norman|Land/geomatics su...|1990-06-24| jturner@example.net|    873.820.0518x825|131900.0|           10|
|      John|    Monroe|        Retail buyer|1968-06-16|  erik33@example.net|    820-813-0557x624|485506.0|            1|
|  Michelle|   Elliott|      Air cabin crew|1975-03-31|tiffanyjohnston@e...|       (705)900-5337|604738.0|            8|
|    Ashley|   Montoya|        C

In [46]:
#adding the skew for department_id=2
emp1 = emp.drop("job_title","email","phone")

emp_df = emp1 #.union(emp1.where("department_id == 2")).union(emp1.where("department_id == 2")).union(emp1.where("department_id == 2")) \
             #.union(emp1.where("department_id == 2")).union(emp1.where("department_id == 2")).union(emp1.where("department_id == 2"))
            # .union(emp.where("department_id == 2")).union(emp.where("department_id == 2")).union(emp.where("department_id == 2"))

In [47]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, avg, sum, min, max, dense_rank, desc

# Window function: rank events by date per user
window_spec = Window.partitionBy("department_id").orderBy(desc("salary"))

df_with_rank = emp_df.withColumn("dense_rank", dense_rank().over(window_spec))


In [48]:
#write to noop - 

df_with_rank.write.format("noop").mode("overwrite").save()

In [8]:
#some sample data with row_number

df_with_rank.where("department_id == 2").where("dense_rank < 10").show()

+----------+---------+----------+--------+-------------+----------+
|first_name|last_name|       dob|  salary|department_id|dense_rank|
+----------+---------+----------+--------+-------------+----------+
|   Richard|   Foster|1990-03-26|999997.0|            2|         1|
|   Richard|   Foster|1990-03-26|999997.0|            2|         1|
|   Cynthia|    Lewis|1994-05-15|999987.0|            2|         2|
|   Cynthia|    Lewis|1994-05-15|999987.0|            2|         2|
|    Melody| Reynolds|1987-09-11|999964.0|            2|         3|
|    Melody| Reynolds|1987-09-11|999964.0|            2|         3|
|    Thomas|   Medina|1997-08-10|999903.0|            2|         4|
|    Thomas|   Medina|1997-08-10|999903.0|            2|         4|
|    Andrea|   Gordon|1998-06-07|999885.0|            2|         5|
|    Andrea|   Gordon|1998-06-07|999885.0|            2|         5|
|    Donald|    Jones|1977-03-29|999875.0|            2|         6|
|    Donald|    Jones|1977-03-29|999875.0|      

In [ ]:
#Please check the Spark Ui -> SQL -> Job -> Stage -> Task details
#Observe the dataSkew Issues
#Observe the DataSpill Issues

In [9]:
# Check the partition count
from pyspark.sql.functions import spark_partition_id, count, lit, desc

part_df = emp_df.withColumn("partition_num", spark_partition_id()).groupBy("partition_num").agg(count(lit(1)).alias("count"))

part_df.orderBy(desc("partition_num")).show()

+-------------+------+
|partition_num| count|
+-------------+------+
|           19|  8713|
|           18| 21699|
|           17| 25986|
|           16| 26064|
|           15| 26090|
|           14| 26114|
|           13| 26128|
|           12| 26275|
|           11| 32533|
|           10| 38454|
|            9| 38519|
|            8| 38449|
|            7| 87281|
|            6|111102|
|            5|111166|
|            4|111166|
|            3|111129|
|            2|111145|
|            1|111175|
|            0|111177|
+-------------+------+



In [13]:
# Verify Employee data based on department_id
from pyspark.sql.functions import count, lit, desc, col

emp_df.groupBy("department_id").agg(count(lit(1))).show()


+-------------+--------+
|department_id|count(1)|
+-------------+--------+
|            1|   99451|
|            6|   99706|
|            3|  100248|
|            5|  200420|
|            9|  100014|
|            4|  100214|
|            8|  100417|
|            7|   99805|
|           10|   99780|
|            2|  200310|
+-------------+--------+



 Dense_rank()

In [50]:
# Step 1: Get distinct (department_id, salary)
salary_distinct = emp_df.select("department_id", "salary").distinct()


In [52]:
# Step 2: Use dense_rank() over department_id, order by salary desc
# This is safe because it's only over distinct values => no skew

drank_window = Window.partitionBy("department_id").orderBy(col("salary").desc())
salary_drank_df = salary_distinct.withColumn("rank", dense_rank().over(drank_window))
salary_drank_df.show()

+-------------+--------+----+
|department_id|  salary|rank|
+-------------+--------+----+
|            1|999996.0|   1|
|            1|999979.0|   2|
|            1|999959.0|   3|
|            1|999922.0|   4|
|            1|999917.0|   5|
|            1|999914.0|   6|
|            1|999911.0|   7|
|            1|999901.0|   8|
|            1|999896.0|   9|
|            1|999894.0|  10|
|            1|999885.0|  11|
|            1|999879.0|  12|
|            1|999863.0|  13|
|            1|999849.0|  14|
|            1|999846.0|  15|
|            1|999844.0|  16|
|            1|999843.0|  17|
|            1|999829.0|  18|
|            1|999815.0|  19|
|            1|999809.0|  20|
+-------------+--------+----+
only showing top 20 rows



In [53]:
# Step 3: Join back to assign rank to full dataset

final_drank_df = emp_df.join(salary_drank_df, on=["department_id", "salary"], how="inner") \
              .select("*") \
              .orderBy("department_id", "rank")

In [54]:
#write to noop - takes 8 Sec

final_drank_df.write.format("noop").mode("overwrite").save()

In [24]:
#some sample data with row_number

final_drank_df.where("department_id == 2").where("rank < 10").show()

+-------------+--------+----------+---------+----------+----+
|department_id|  salary|first_name|last_name|       dob|rank|
+-------------+--------+----------+---------+----------+----+
|            2|999997.0|   Richard|   Foster|1990-03-26|   1|
|            2|999997.0|   Richard|   Foster|1990-03-26|   1|
|            2|999987.0|   Cynthia|    Lewis|1994-05-15|   2|
|            2|999987.0|   Cynthia|    Lewis|1994-05-15|   2|
|            2|999964.0|    Melody| Reynolds|1987-09-11|   3|
|            2|999964.0|    Melody| Reynolds|1987-09-11|   3|
|            2|999903.0|    Thomas|   Medina|1997-08-10|   4|
|            2|999903.0|    Thomas|   Medina|1997-08-10|   4|
|            2|999885.0|    Andrea|   Gordon|1998-06-07|   5|
|            2|999885.0|    Andrea|   Gordon|1998-06-07|   5|
|            2|999875.0|    Donald|    Jones|1977-03-29|   6|
|            2|999875.0|    Donald|    Jones|1977-03-29|   6|
|            2|999873.0|     Tammy|   Finley|1966-10-08|   7|
|       

In [40]:
#some sample data with row_number

df_with_rank.where("department_id == 2").where("dense_rank < 10").show()

+----------+---------+----------+--------+-------------+----------+
|first_name|last_name|       dob|  salary|department_id|dense_rank|
+----------+---------+----------+--------+-------------+----------+
|   Richard|   Foster|1990-03-26|999997.0|            2|         1|
|   Richard|   Foster|1990-03-26|999997.0|            2|         1|
|   Cynthia|    Lewis|1994-05-15|999987.0|            2|         2|
|   Cynthia|    Lewis|1994-05-15|999987.0|            2|         2|
|    Melody| Reynolds|1987-09-11|999964.0|            2|         3|
|    Melody| Reynolds|1987-09-11|999964.0|            2|         3|
|    Thomas|   Medina|1997-08-10|999903.0|            2|         4|
|    Thomas|   Medina|1997-08-10|999903.0|            2|         4|
|    Andrea|   Gordon|1998-06-07|999885.0|            2|         5|
|    Andrea|   Gordon|1998-06-07|999885.0|            2|         5|
|    Donald|    Jones|1977-03-29|999875.0|            2|         6|
|    Donald|    Jones|1977-03-29|999875.0|      

✅ Summary:
    
1. With this approch we were able to solve this problem with 60% less resources and improved Job execution time by 80%
2. If you try with larger datasets, you will definatly see performance improvements
